In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('data/raw_data/apt_info_data_건설사수정.csv', encoding='cp949')

### 네이버 부동산 아파트 단지 정보 데이터

#### 불필요한 컬럼 제거

In [3]:
data.drop([
    '면적','최저층','난방','공급면적','해당면적_세대수',
    '재산세','재산세합계','지방교육세','재산세_도시지역분','종합부동산세',
    '결정세액','농어촌특별세','가격','겨울관리비','여름관리비',
    '매매호가','전세호가','월세호가','실거래가'], axis=1, inplace=True)

#### 전처리 필요한 컬럼들 처리

In [4]:
for col in ['방수','욕실수']: data[col] = data[col].replace('-',0).astype(int)

#### 일정 이상 전용면적, 세대수만 사용

In [5]:
data = data[(data['전용면적']>=40) & (data['세대수']>=100)].reset_index(drop=True)

In [6]:
data.shape

(84238, 21)

#### 기존 컬럼 가공

- (신규) 임대세대비율 : 임대세대수 / 세대수
- (신규) 세대당_주차대수 : 주차대수 / 세대수
- 현관구조 : label encoding
- 초등학교_설립정보 : label encoding
- 초등학교_학생수 : 남학생수 + 여학생수

In [7]:
data['현관구조'].factorize()

(array([0, 0, 0, ..., 0, 0, 0], dtype=int64),
 Index(['계단식', '복합식', '복도식'], dtype='object'))

In [8]:
data['초등학교_설립정보'].factorize()

(array([ 0,  0,  0, ..., -1, -1, -1], dtype=int64),
 Index(['공립', '혁신'], dtype='object'))

In [9]:
data['임대세대비율'] = data['임대세대수'] / data['세대수']
data['세대당_주차대수'] = data['주차대수'] / data['세대수']
data['현관구조'] = data['현관구조'].factorize()[0]
data['초등학교_설립정보'] = data['초등학교_설립정보'].factorize()[0]
data['초등학교_학생수'] = data['초등학교_남학생수'].fillna(0) + data['초등학교_여학생수'].fillna(0)

- 전용면적구간: 60 미만, 75 미만, 85 미만, 103 미만, 103 이상으로 분류

In [10]:
for i in range(4): data[f'temp_{i}'] = data['전용면적'] >= [60, 75, 85, 103][i]
data['전용면적구간'] = data[[f'temp_{i}' for i in range(4)]].sum(axis=1)
data.drop([f'temp_{i}' for i in range(4)], axis=1, inplace=True)
data.drop(['임대세대수','주차대수','초등학교_남학생수','초등학교_여학생수','전용면적'], axis=1, inplace=True)
data.reset_index(drop=True, inplace=True)

- 건설사: 시공능력평가 10위 이내 건설사인지 아닌지 구분
- 삼성물산, 현대건설, 대우건설, 현대엔지니어링, GS건설, DL이앤씨, 포스코이앤씨, 롯데건설, SK에코플랜트, 호반건설
- 별도의 노트북에 작성

#### 중복된 주소 있는지 체크

- 도로명주소, 법정동주소 중복되는 아파트 각각 약 200여개씩 존재

- 대부분 하나의 아파트단지로 봐도 무방한 경우

- 도로명주소가 잘못된 경우 법정동 주소 사용

In [11]:
idx = data[data['도로명주소'].isin(['대구시 동구', '대구시 수성구'])].index
data.loc[idx,'도로명주소'] = data.loc[idx, '법정동주소'].str.replace('[번지|일원]','', regex=True).apply(lambda x: x.strip())
data = data.drop_duplicates(['도로명주소','전용면적구간']).reset_index(drop=True)
data.shape

(39811, 20)

#### 지하철 데이터 결합

- 하버사인 공식 이용하여 아파트 단지와 역 과의 거리 산출
- 직접 역세권: 500m이내, 간접 역세권: 1km 이내
- 값: 노선 개수만 활용, 나머지는 디버그용

In [12]:
sub_data = pd.read_csv('data/subway.csv', encoding='cp949')

In [13]:
def haversine_np(lat1,lon1,lat2,lon2):
    lat1,lon1,lat2,lon2 = map(np.radians, [lat1,lon1,lat2,lon2])
    dlon,dlat = lon2-lon1,lat2-lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return 6371000*c

In [14]:
table = data[['도로명주소', 'latitude', 'longitude']].drop_duplicates().reset_index(drop=True)
arr = []
for idx, row in table.iterrows():
    lat1, lon1 = row['latitude'], row['longitude']
    dist = haversine_np(lat1, lon1, sub_data['위도'], sub_data['경도'])
    tmp = []
    for i in range(2):
        j = dist[dist<=[500,1000][i]].index
        tmp += [
            sub_data.loc[j,'호선'].nunique(), 
            *map(lambda x: ', '.join(sorted(sub_data.loc[j][x].unique())),['호선','역사명'])
            ]
    arr += tmp
    print(f'\r{idx+1}/{table.shape[0]} completed.', end='')

84/19671 completed.

19671/19671 completed.

In [15]:
cols = ['직접역세권_노선수', '직접역세권_노선', '직접역세권_역', 
        '간접역세권_노선수', '간접역세권_노선', '간접역세권_역']
table[cols] = np.array(arr).reshape(-1,6)
data = data.merge(table, on=['도로명주소','latitude','longitude'])

#### 고속철 데이터 결합

- 직접 역세권: 1km 이내, 간접 역세권: 5km 이내
- 범위 내에 역이 존재하는지 여부를 활용 (0: 해당사항없음, 1: 간접역세권, 2: 직접역세권)
- 역명: 디버그용

In [16]:
train_data = pd.read_csv('data/train.csv', encoding='cp949')

In [17]:
table = data[['도로명주소', 'latitude', 'longitude']].drop_duplicates().reset_index(drop=True)
arr = []
for idx, row in table.iterrows():
    lat1, lon1 = row['latitude'], row['longitude']
    dist = haversine_np(lat1, lon1, train_data['위도좌표'], train_data['경도좌표'])
    tmp = [0]
    for i in range(2):
        j = dist[dist<=[1000,5000][i]].index
        tmp[0] += len(j)>0
        tmp += [', '.join(sorted(train_data.loc[j,'역이름'].unique()))]
    arr += tmp
    print(f'\r{idx+1}/{table.shape[0]} completed.', end='')

19671/19671 completed.

In [18]:
table[['고속철_직접역세권_여부', '고속철_직접역세권_역명', '고속철_간접역세권_역명']] = np.array(arr).reshape(-1,3)
data = data.merge(table, on=['도로명주소','latitude','longitude'])

#### 중학교 데이터 결합

In [19]:
ms_data = pd.read_csv('data/ms_data.csv', encoding='cp949')

- 위,경도 이상치 직접 수정

- 가장 가까운 중학교의 학생 수 및 평균 학업성취도
- 만약 일정 거리 이내에 중학교가 없을 시, 가장 가까운 학교를 사용할 것인지 결측치로 둘 것 인지

In [20]:
table = data[['도로명주소', 'latitude', 'longitude']].drop_duplicates().reset_index(drop=True)
arr = []
for idx, row in table.iterrows():
    lat1, lon1 = row['latitude'], row['longitude']
    dist = haversine_np(lat1, lon1, ms_data['위도'], ms_data['경도'])
    arr += [ms_data.iloc[dist.argmin(),[0,3,4]].to_list()+[min(dist)]]
    print(f'\r{idx+1}/{table.shape[0]} completed.', end='')

12/19671 completed.

19671/19671 completed.

In [21]:
table[['중학교명', '중학교_학업성취도', '중학교_졸업자수', '중학교_최단거리']] = np.array(arr)
data = data.merge(table, on=['도로명주소','latitude','longitude'])

#### 초등학교 데이터 결합
- 일단 네이버 부동산 정보에 존재하는 초등학교들과 초등학교 데이터셋의 좌표가 잘 맞는지 확인
- 이상이 있는 초등학교들의 좌표들을 수정한 후 모든 아파트에 대해 가장 거리가 짧은 초등학교를 산출

In [22]:
es_data = pd.read_csv('data/raw_data/2023년도_학교기본정보(초)_전체.csv')

- 잘못된 위,경도 데이터 수정

In [23]:
es_data[es_data['위도'].isna()][['지역', '학교명']]

,지역,학교명
987,대구광역시 수성구,대구노변초등학교
1070,대구광역시 서구,대구비산초등학교
1089,대구광역시 서구,대구평리초등학교
1910,경기도 가평군,미원초등학교위곡분교장
3248,경기도 광주시,쌍동초등학교
3249,경기도 평택시,율포초등학교
3607,강원특별자치도 춘천시,성원초등학교
4226,충청남도 태안군,안흥초등학교
4310,충청남도 아산시,아산갈산초등학교
5804,경상남도 창원시 마산합포구,구산초등학교


In [24]:
idx = es_data[es_data['위도'].isna()].index
es_data.loc[idx, ['위도','경도']] = [
    [35.8376255,128.6975583],[35.8789898,128.5639202],
    [35.872413,128.5608249],[37.6706524,127.5152904],
    [37.3777078,127.2896552],[37.0424627,127.043416],
    [37.8479864,127.7448595],[36.6955459,126.1744853],
    [36.7937819,127.0687703],[35.1202469,128.5793284]
]

In [25]:
es_data[(es_data['위도']>39) | (es_data['위도']<33)][['지역','학교명','위도','경도']]

,지역,학교명,위도,경도
984,대구광역시 수성구,대구고산초등학교,39.200000,134.600000
4075,충청남도 천안시 서북구,천안미라초등학교,86.801078,127.128405
5444,경상북도 영양군,석보초등학교,12.907232,363.329600


In [26]:
idx = es_data[(es_data['위도']>39) | (es_data['위도']<33)].index
es_data.loc[idx, ['위도','경도']] = [
    [35.8443168,128.6931978],[36.8008807,127.1284485],[36.5582246,129.1231255]
]

In [27]:
es_data[(es_data['경도']>131) | (es_data['경도']<124)][['지역','학교명','위도','경도']]

,지역,학교명,위도,경도
3104,경기도 화성시,하길초등학교,37.098516,922.319000
3226,경기도 하남시,신우초등학교,37.513210,112.716005


In [28]:
idx = es_data[(es_data['경도']>131) | (es_data['경도']<124)].index
es_data.loc[idx, ['위도','경도']] = [
    [37.115561,126.9040217],[37.5110769,127.1611818]
]

- 위, 경도 데이터 결합하여 거리 계산

In [29]:
es_data['학교도로명 주소'] = es_data['학교도로명 주소'].fillna('충청남도 당진시')
es_data['지역'] = es_data['학교도로명 주소'].apply(lambda x: ' '.join(x.split()[:2-x.startswith('세종')]))
es_data['학교명'] = es_data['지역'] + ' '+ es_data['학교명']
for s in ['특별','자치','광역']: es_data['학교명'] = es_data['학교명'].str.replace(s,'')
for c in ['서울', '광주', '대전', '울산']: 
    es_data['학교명'] = es_data['학교명'].str.replace(c+' ', c+'시 ')
for k,v in {'경기 ':'경기도 ', '전북 ':'전라북도 ', '경남 ':'경상남도 '}.items():
    es_data['학교명'] = es_data['학교명'].str.replace(k,v)

In [30]:
nav_es = data[['아파트명','법정동주소','도로명주소','초등학교_학군정보','latitude','longitude']].drop_duplicates('초등학교_학군정보').dropna()
nav_es['초등학교명'] = nav_es['도로명주소'].apply(lambda x: ' '.join(x.split()[:2-x.startswith('세종')])) + ' ' + nav_es['초등학교_학군정보']

In [31]:
diff = [x.split()[-1] for x in (set(nav_es['초등학교명']) - set(es_data['학교명']))]
es_data['학교명'] = es_data['학교명'].apply(lambda x: ' '.join(x.split()[::2]) if x.split()[-1] in diff else x)
nav_es['초등학교명'] = nav_es['초등학교명'].apply(lambda x: ' '.join(x.split()[::2]) if x.split()[-1] in diff else x)

In [32]:
diff = set(nav_es['초등학교명']) - set(es_data['학교명'])
diff

{'경기도 서울누원초등학교', '부산시 위봉초등학교', '부산시 좌성초등학교', '서울시 서울반포초등학교', '전라북도 군산아리울초등학교'}

서울누원초등학교 -> 경기도에서 서울로 수정

서울반포초등학교 -> 재건축 때문에 휴교중, 수동으로 추가

나머지 -> 폐교됨

In [33]:
nav_es['초등학교명'] = nav_es['초등학교명'].replace('경기도 서울누원초등학교', '서울시 서울누원초등학교')
idx = es_data[es_data['학교명'].str.contains('아리울')].index
es_data.loc[idx, '학교명'] = '전라북도 군산아리울초등학교'
es_data = es_data[['학교명','위도', '경도','학교도로명 주소']]
es_data.columns = ['초등학교명', '초등학교_위도', '초등학교_경도', '학교도로명_주소']
es_data.loc[len(es_data)] = ['서울시 서초구 서울반포초등학교', 37.5027, 126.9911, '서울특별시 서초구 신반포로 55-4']

In [34]:
nav_es = nav_es.merge(es_data, on='초등학교명', how='left').reset_index(drop=True)
nav_es['거리'] = haversine_np(nav_es['latitude'], nav_es['longitude'], nav_es['초등학교_위도'], nav_es['초등학교_경도'])

학교 위경도가 잘못되어 거리가 비정상적으로 잡히는 학교들이 다수 존재

-> 거리 1km이상 학교들만 카카오 api 이용하여 도로명주소를 위경도로 변환하여 다시 거리 계산

In [35]:
anomaly = pd.read_csv('data/es_data_anomaly.csv')

In [36]:
anomaly['초등학교_거리'] = haversine_np(anomaly['latitude'], anomaly['longitude'], anomaly['lat'], anomaly['lon'])
anomaly[anomaly['초등학교_거리'].isna()]

,Unnamed: 0,아파트명,법정동주소,도로명주소,초등학교_학군정보,latitude,longitude,초등학교명,초등학교_위도,초등학교_경도,학교도로명_주소,거리,lon,lat,초등학교_거리
146,146,금화마을3단지주공그린빌,경기도 용인시 기흥구 상갈동 481,경기도 용인시 기흥구 금화로11번길 10,보라초등학교,37.262336,127.107341,경기도 용인시 보라초등학교,37.3,127.1,경기도 용인시 기흥구 금화로 105,4238.109048,NaN,NaN,NaN


In [37]:
anomaly['초등학교_거리'].fillna(haversine_np(37.262336, 127.107341, 37.2639233, 127.1072747), inplace=True)

In [38]:
anomaly = anomaly[['lat','lon','학교도로명_주소']]
es_data = es_data.merge(anomaly, on='학교도로명_주소', how='left')
es_data['lat'].fillna(es_data['초등학교_위도'], inplace=True)
es_data['lon'].fillna(es_data['초등학교_경도'], inplace=True)
es_data
#es_data = es_data[['초등학교명, 학교도로명_주소', 'lat', 'lon']]

,초등학교명,초등학교_위도,초등학교_경도,학교도로명_주소,lat,lon
0,서울시 서초구 서울교육대학교부설초등학교,37.490739,127.015424,서울특별시 서초구 서초중앙로 96,37.490739,127.015424
1,서울시 종로구 서울대학교사범대학부설초등학교,37.577017,127.003091,서울특별시 종로구 대학로 64,37.577017,127.003091
2,서울시 강남구 서울개일초등학교,37.486214,127.057742,서울특별시 강남구 개포로 401,37.486214,127.057742
3,서울시 강남구 서울개포초등학교,37.486805,127.069977,서울특별시 강남구 삼성로4길 30,37.486805,127.069977
4,서울시 강남구 서울구룡초등학교,37.481193,127.051755,서울특별시 강남구 개포로 263,37.481193,127.051755
...,...,...,...,...,...,...
6315,제주도 제주시 이도초등학교,33.488121,126.531991,제주특별자치도 제주시 구남동2길 58,33.488121,126.531991
6316,제주도 제주시 도련초등학교,33.514019,126.584301,제주특별자치도 제주시 화삼로 104,33.514019,126.584301
6317,제주도 제주시 삼화초등학교,33.515224,126.577005,제주특별자치도 제주시 화삼북로2길 27,33.515224,126.577005
6318,제주도 제주시 하귀일초등학교,33.483420,126.415364,제주특별자치도 제주시 애월읍 하귀9길 42,33.483420,126.415364


In [39]:
es_data.isna().sum()

초등학교명       0
초등학교_위도     0
초등학교_경도     0
학교도로명_주소    0
lat         0
lon         0
dtype: int64

In [40]:
table = data[['도로명주소', 'latitude', 'longitude']].drop_duplicates().reset_index(drop=True)
arr = []
for idx, row in table.iterrows():
    lat1, lon1 = row['latitude'], row['longitude']
    dist = haversine_np(lat1, lon1, es_data['lat'], es_data['lon'])
    arr += [es_data.iloc[dist.argmin(),[0,3]].to_list()+[min(dist)]]
    print(f'\r{idx+1}/{table.shape[0]} completed.', end='')

194/19671 completed.

19671/19671 completed.

In [41]:
table[['초등학교명', '초등학교_도로명주소', '초등학교_최단거리']] = np.array(arr)
table['초등학교_최단거리'] = table['초등학교_최단거리'].astype(float)
for i in range(4): table[f'temp_{i}'] = table['초등학교_최단거리'] >= [200, 500, 1000, 2000][i]
table['초등학교_최단거리구간'] = table[[f'temp_{i}' for i in range(4)]].sum(axis=1)
table.drop([f'temp_{i}' for i in range(4)], axis=1, inplace=True)
data = data.merge(table, on=['도로명주소','latitude','longitude'], how='left')

In [42]:
data.shape

(39811, 37)

#### 스타벅스 매장

- 도보거리 (500m) 이내에 있는 스타벅스 매장의 개수

In [43]:
sb_data = pd.read_csv('data/starbucks_231211.csv', encoding='cp949')

In [44]:
sb_data = sb_data[['s_name', 'lat','lot']]

In [45]:
table = data[['도로명주소', 'latitude', 'longitude']].drop_duplicates().reset_index(drop=True)
arr = []
for idx, row in table.iterrows():
    lat1, lon1 = row['latitude'], row['longitude']
    dist = haversine_np(lat1, lon1, sb_data['lat'], sb_data['lot'])
    j = dist[dist<=500].index
    arr += [len(j), ', '.join(sorted(sb_data.loc[j,'s_name'].unique()))]
    print(f'\r{idx+1}/{table.shape[0]} completed.', end='')

19671/19671 completed.

In [46]:
table[['스타벅스_매장수', '스타벅스_매장이름']] = np.array(arr).reshape(-1,2)
data = data.merge(table, on=['도로명주소','latitude','longitude'], how='left')

### 쇼핑몰, 대형마트

- 백화점 및 쇼핑몰: 5km 이내 존재 여부
- 대형마트: 500m / 3km 이내 존재 여부

In [49]:
dm_data = pd.read_csv('data/쇼핑몰_마트_최종.csv', encoding='cp949')

In [50]:
d_data, m_data = dm_data[dm_data['업태구분명']!='대형마트'].reset_index(drop=True), dm_data[dm_data['업태구분명']=='대형마트'].reset_index(drop=True)

In [51]:
table = data[['도로명주소', 'latitude', 'longitude']].drop_duplicates().reset_index(drop=True)
arr = []
for idx, row in table.iterrows():
    lat1, lon1 = row['latitude'], row['longitude']
    dist = haversine_np(lat1, lon1, d_data['lat'], d_data['lon'])
    tmp = [0]
    j = dist[dist<=5000].index
    tmp[0] = (len(j)>0)+0
    tmp += [d_data.loc[dist.argmin(),'사업장명'], dist.min()]
    arr += tmp
    print(f'\r{idx+1}/{table.shape[0]} completed.', end='')

19671/19671 completed.

In [52]:
table[['백화점쇼핑몰_유무', '백화점쇼핑몰_매장이름', '백화점쇼핑몰_최단거리']] = np.array(arr).reshape(-1,3)
table['백화점쇼핑몰_유무'] = table['백화점쇼핑몰_유무'].astype(int)
table['백화점쇼핑몰_최단거리'] = table['백화점쇼핑몰_최단거리'].astype(float)
data = data.merge(table, on=['도로명주소','latitude','longitude'], how='left')

In [53]:
table = data[['도로명주소', 'latitude', 'longitude']].drop_duplicates().reset_index(drop=True)
arr = []
for idx, row in table.iterrows():
    lat1, lon1 = row['latitude'], row['longitude']
    dist = haversine_np(lat1, lon1, m_data['lat'], m_data['lon'])
    tmp = [0]
    for i in range(2):
        j = dist[dist<=[500,3000][i]].index
        tmp[0] += len(j)>0
    tmp += [m_data.loc[dist.argmin(),'사업장명'], dist.min()]
    arr += tmp
    print(f'\r{idx+1}/{table.shape[0]} completed.', end='')

19671/19671 completed.

In [54]:
table[['마트_유무', '마트_매장이름', '마트_최단거리']] = np.array(arr).reshape(-1,3)
table['마트_유무'] = table['마트_유무'].astype(int)
table['마트_최단거리'] = table['마트_최단거리'].astype(float)
data = data.merge(table, on=['도로명주소','latitude','longitude'], how='left')

#### 중심업무지구, 산업단지

- 기준: 2km / 6km 

In [55]:
bd_data = pd.concat([pd.read_csv('data/중심업무지구.csv'), pd.read_csv('data/산업단지.csv', encoding='cp949')]).reset_index(drop=True)

In [56]:
table = data[['도로명주소', 'latitude', 'longitude']].drop_duplicates().reset_index(drop=True)
arr = []
for idx, row in table.iterrows():
    lat1, lon1 = row['latitude'], row['longitude']
    dist = haversine_np(lat1, lon1, bd_data['위도'], bd_data['경도'])
    tmp = [0]
    for i in range(2):
        j = dist[dist<=[2000,6000][i]].index
        tmp[0] += len(j)>0
    tmp += [bd_data.loc[dist.argmin(),'이름'], dist.min()]
    arr += tmp
    print(f'\r{idx+1}/{table.shape[0]} completed.', end='')

19671/19671 completed.

In [57]:
table[['직주_근접도', '직주_이름', '직주_최단거리']] = np.array(arr).reshape(-1,3)
table['직주_근접도'] = table['직주_근접도'].astype(int)
table['직주_최단거리'] = table['직주_최단거리'].astype(float)
data = data.merge(table, on=['도로명주소','latitude','longitude'], how='left')

#### 학원가

In [61]:
ac_data = pd.read_csv('data/academy.csv', usecols=[1,3,4,5]).dropna().reset_index(drop=True)
ac_data.columns = ['학원가', '도로명주소', 'lon', 'lat']

In [62]:
table = data[['도로명주소', 'latitude', 'longitude']].drop_duplicates().reset_index(drop=True)
arr = []
for idx, row in table.iterrows():
    lat1, lon1 = row['latitude'], row['longitude']
    dist = haversine_np(lat1, lon1, ac_data['lat'], ac_data['lon'])
    tmp = [0]
    for i in range(2):
        j = dist[dist<=[1000,3000][i]].index
        tmp[0] += len(j)>0
    tmp += [ac_data.loc[dist.argmin(),'학원가'], dist.min()]
    arr += tmp
    print(f'\r{idx+1}/{table.shape[0]} completed.', end='')

19671/19671 completed.

In [63]:
cols = ['학원가_근접도', '최근접학원가', '학원가_최단거리']
table[cols] = np.array(arr).reshape(-1,3)
data = data.merge(table, on=['도로명주소','latitude','longitude'])

In [64]:
data.to_csv('data/apt_info_data.csv', encoding='cp949', index=False)